In [1]:
from transformers import VitsTokenizer, VitsModel
import numpy as np
import torch
import scipy
from IPython.display import Audio

In [2]:
# Load the Bengali tokenizer and model
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-ben")
model = VitsModel.from_pretrained("facebook/mms-tts-ben")

Some weights of the model checkpoint at facebook/mms-tts-ben were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [3]:
# Set the device for computation (GPU if available, otherwise CPU)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device).eval()

VitsModel(
  (text_encoder): VitsTextEncoder(
    (embed_tokens): Embedding(74, 192)
    (encoder): VitsEncoder(
      (layers): ModuleList(
        (0-5): 6 x VitsEncoderLayer(
          (attention): VitsAttention(
            (k_proj): Linear(in_features=192, out_features=192, bias=True)
            (v_proj): Linear(in_features=192, out_features=192, bias=True)
            (q_proj): Linear(in_features=192, out_features=192, bias=True)
            (out_proj): Linear(in_features=192, out_features=192, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (feed_forward): VitsFeedForward(
            (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
            (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
            (dropout): Dropout(p=0.1, inplace=False)
            (act_fn): ReLU()
          )
          (final_layer_norm): LayerNorm((192,), eps=1e-05, eleme

In [4]:
# Input text for TTS
text = "এই প্রকল্পটি আমাকে নতুন প্রযুক্তি অন্বেষণ এবং আমার দক্ষতা বাড়াতে সাহায্য করেছে।"

In [5]:
# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

In [6]:
# Set a random seed for reproducibility
from transformers import set_seed
set_seed(555)

2024-10-15 11:45:20.583340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 11:45:20.793565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 11:45:20.868673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 11:45:21.198197: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 11:45:23.556625: W tensorflow/compiler/tf2

In [7]:
# Generate the waveform without tracking gradients
with torch.no_grad():
    outputs = model(**inputs.to(device))

In [8]:
# Extract the waveform and ensure it's a 1D numpy array
waveform = outputs.waveform[0].cpu().float().numpy()
if waveform.ndim > 1:
    waveform = waveform.flatten()  # Flatten if it's multi-dimensional